In [ ]:
#Initailize the LLM with Groq
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()  # loads environment variables from .env

llm = ChatGroq(model_name="openai/gpt-oss-20b", api_key=os.getenv("GROQ_API_KEY"))

llm.invoke("Hello, diff between cursor vs vscode?")


In [ ]:
#Initailize the LLM with OpenAI
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI#llm creation
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get("OPEN_API_KEY")

#llm creation
openAillm = ChatOpenAI(model_name="gpt-3.5-turbo")
openAillm.invoke("Hello, diff between cursor vs vscode?")

In [ ]:
# define the tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper_wiki = WikipediaAPIWrapper(tool_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
##To test run the wiki tool
# print(wiki.invoke("hello world program"))
llm_with_tools = llm.bind_tools([wiki])

In [ ]:
#lang graph imports
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage ## human or ai messages
from typing import Annotated ## for labelling the messages
from langgraph.graph.message import add_messages ## reducers for messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display ## for graph visualization
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
## lang graph initializ

## langgraph state creation
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages] 

## Node creation
def llm_node(state: State) -> State:
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

## graph creation
graph_builder = StateGraph(State)

# add nodes to the graph
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", ToolNode(tools=[wiki]))

# add edges to the graph
graph_builder.add_edge(START, "llm")
graph_builder.add_conditional_edges(
    "llm",
    tools_condition,
    {
        "tools": "tools",
        "__end__": END
    }
)
graph_builder.add_edge("tools", "llm")

## by default the memory will be shared in only one session (starting the graph => end graph)
#graph = graph_builder.compile()

# to make the memory across the sessions need (memory = MemorySaver())
memory = MemorySaver()
graph = graph_builder.compile(memory)

#View the graph
display(Image(graph.get_graph().draw_mermaid_png()))
    

In [ ]:
#### if the below code throws an error, uncomment graph = graph_builder.compile()
#### if you invoke with without memory saver the below example will not work (expected output will not get)
messages = graph.invoke({"messages": "tell me about tamil language"});

for msg in messages["messages"]:
    msg.pretty_print()

#now we are invoking the llm with new session, so existing doesn't know,it will not answer
messages = graph.invoke({"messages": "how old is?"});
print("*******************************")
for msg in messages["messages"]:
    msg.pretty_print()

In [ ]:
#### if the below code throws an error, uncomment (memory = MemorySaver() graph = graph_builder.compile(memory) )
#### this will work just opposite to above example, the context will be shared for across the session based on session_id
thread_id = "session_001"
messages = graph.invoke({"messages": "tell me about tamil language"}, config={"thread_id": thread_id});

for msg in messages["messages"]:
    msg.pretty_print()

#now we are invoking the llm with new session, so existing doesn't know,it will not answer
messages = graph.invoke({"messages": "how old is?"}, config={"thread_id": thread_id});
print("*******************************")
for msg in messages["messages"]:
    msg.pretty_print()

In [ ]:

#cli chatbot
thread_id = "session_002"
while True:
    input_msg = input("You: ")
    if input_msg in {"quit", "exit"}:
        break
    response = graph.invoke({"messages": input_msg}, config={"thread_id": thread_id})
    chat_history = response["messages"][-1]
    chat_history.pretty_print()